In [4]:
import time
import roboticstoolbox as rtb
from roboticstoolbox import ERobot, ET, ETS
from spatialmath import SE3
from spatialgeometry import Mesh
from scipy.spatial.transform import Rotation as R
from itertools import product
import numpy as np
from spatialmath.base import transl

# Define the sequence of rotation axes for the joints
axes = ['z', 'x', 'z', 'x', 'z', 'x']
lengths = [0.2, 0.2, 0.2, 0.2, 0.2, 0.2]
# Initialize an empty ETS
ets = ETS()

transform_dict = {
    'z': ET.Rz,
    'x': ET.Rx,
    'y': ET.Ry
}


# define inertia tensor for cylinder
def inertia_tensor_cylinder(mass, radius, height):
    # Inertia tensor for a solid cylinder about its central axis
    I_xx = (1/12) * mass * (3 * radius**2 + height**2)
    I_yy = (1/12) * mass * (3 * radius**2 + height**2)
    I_zz = (0.5) * mass * radius**2
    return np.array([[I_xx, 0, 0],
                     [0, I_yy, 0],
                     [0, 0, I_zz]])


# Build the ETS by appending a variable rotation about each specified axis
for length, axis in zip(lengths, axes):
    ets *= transform_dict[axis]()
    ets *= ET.tz(length)

# Create the robot model from the ETS
robot = ERobot(ets, name='6-Link Robot Arm')

initial = np.array([0, 100, 0])
z_range = np.linspace(np.radians(-70), np.radians(70), 30) #left/right rotation
y_range = np.linspace(np.radians(-40), np.radians(40), 30) #neck lateral bending
x_range = np.linspace(np.radians(-60), np.radians(60), 30) #neck extension/ flexion

prod = np.array(list(product(z_range, y_range, x_range)))

rotations = R.from_euler('zyx', prod, degrees=False)
rotated = rotations.apply(initial)

zipper = zip(rotations, rotated)

import spatialmath.base as base
q = base.sym.symbol("q_:6")
# do inverese dynamics    
robot.manipulability(q=np.zeros(6), method='asada')
# robot.jacobe(q)

0.0

In [13]:
robot.links[0].r = [0, 0, 0.1]
robot.links[0].I = inertia_tensor_cylinder(5, 0.1, 0.4)
robot.links[0].m = 5
robot.links[1].
robot

ERobot: 6-Link Robot Arm, 6 joints (RRRRRR), dynamics
┌──────┬────────┬───────┬────────┬─────────────────────┐
│ link │  link  │ joint │ parent │ ETS: parent to link │
├──────┼────────┼───────┼────────┼─────────────────────┤
│    0 │ test   │     0 │ BASE   │ Rz(q0)              │
│    1 │ link1  │     1 │ test   │ tz(0.2) ⊕ Rx(q1)    │
│    2 │ link2  │     2 │ link1  │ tz(0.2) ⊕ Rz(q2)    │
│    3 │ link3  │     3 │ link2  │ tz(0.2) ⊕ Rx(q3)    │
│    4 │ link4  │     4 │ link3  │ tz(0.2) ⊕ Rz(q4)    │
│    5 │ link5  │     5 │ link4  │ tz(0.2) ⊕ Rx(q5)    │
│    6 │ @link6 │       │ link5  │ tz(0.2)             │
└──────┴────────┴───────┴────────┴─────────────────────┘

In [1]:
from roboticstoolbox import DHRobot, RevoluteDH

# Link 1
L1 = RevoluteDH(
    d=0.4,       # Link offset
    a=0.25,      # Link length
    alpha=-np.pi/2,  # Link twist
    m=5.0,       # Mass of the link
    r=[0, 0, 0], # Center of mass position in link frame
    I=[0.1, 0.1, 0.1]  # Inertia tensor components
)

# Repeat for all 6 links
L2 = RevoluteDH(d=0, a=0.3, alpha=0, m=3.0, r=[0, 0, 0], I=[0.05, 0.05, 0.05])
L3 = RevoluteDH(d=0, a=0.2, alpha=-np.pi/2, m=2.0, r=[0, 0, 0], I=[0.02, 0.02, 0.02])
L4 = RevoluteDH(d=0.4, a=0, alpha=np.pi/2, m=1.0, r=[0, 0, 0], I=[0.01, 0.01, 0.01])
L5 = RevoluteDH(d=0, a=0, alpha=-np.pi/2, m=0.5, r=[0, 0, 0], I=[0.005, 0.005, 0.005])
L6 = RevoluteDH(d=0.1, a=0, alpha=0, m=0.2, r=[0, 0, 0], I=[0.002, 0.002, 0.002])

# Create the robot model
robot = DHRobot([L1, L2, L3, L4, L5, L6], name='Custom6DOFRobot')

# Optionally, set gravity (default is [0, 0, 9.81])
robot.gravity = [0, 0, -9.81]  # Gravity in negative Z-direction

NameError: name 'np' is not defined